# 案例: 数据驱动的仿真

本案例演示了如何使用外部数据（来自CSV文件）来驱动一个仿真。一个`CsvDataSourceAgent`智能体将负责读取CSV文件，并将数据发布到消息总线上，一个水库组件将订阅这些数据作为其入流。

### 1. 导入库并加载配置

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

from swp.core_engine.testing.simulation_harness import SimulationHarness
from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.data_access.csv_data_source import CsvDataSourceAgent

CONFIG_PATH = 'examples/data_driven_simulation.json'
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

# 设置 Matplotlib 样式
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

### 2. 定义并运行仿真

In [ ]:
def run_data_driven_simulation(cfg):
    print("\n--- 搭建数据驱动的仿真示例 ---")

    # 1. 仿真和通信设置
    harness = SimulationHarness(config=cfg['simulation'])
    message_bus = harness.message_bus
    data_topic = cfg['messaging']['inflow_data_topic']

    # 2. 创建数据源智能体
    agent_cfg = cfg['agents']['csv_data_source']
    csv_agent = CsvDataSourceAgent(
        agent_id="csv_inflow_agent",
        message_bus=message_bus,
        csv_filepath=agent_cfg['csv_filepath'],
        publish_topic=data_topic
    )

    # 3. 创建物理组件
    res_cfg = cfg['components']['reservoir']
    reservoir = Reservoir(
        name="data_driven_reservoir",
        initial_state=res_cfg['initial_state'],
        parameters=res_cfg['params'],
        message_bus=message_bus,
        inflow_topic=data_topic
    )

    gate_cfg = cfg['components']['gate']
    gate = Gate(
        name="outflow_gate",
        initial_state=gate_cfg['initial_state'],
        parameters=gate_cfg['params']
    )

    # 4. 添加组件和智能体到仿真平台
    harness.add_component(reservoir)
    harness.add_component(gate)
    harness.add_agent(csv_agent)

    # 5. 定义拓扑并构建
    harness.add_connection("data_driven_reservoir", "outflow_gate")
    harness.build()

    # 6. 运行仿真
    harness.run_mas_simulation()

    print("\n--- 仿真结束后的最终状态 ---")
    final_states = {cid: comp.get_state() for cid, comp in harness.components.items()}
    for cid, state in final_states.items():
        state_str = ", ".join(f"{k}={v:.2f}" for k, v in state.items())
        print(f"  {cid}: {state_str}")
        
    return harness.history

history = run_data_driven_simulation(config)

### 3. 可视化结果

我们可以绘制水库水位随时间的变化，以直观地看到CSV数据是如何影响系统状态的。

In [ ]:
def plot_history(simulation_history):
    if not simulation_history:
        print("没有历史数据可供绘制。")
        return
        
    # 将历史记录转换为Pandas DataFrame以便于绘图
    flat_history = []
    for step in simulation_history:
        row = {'time': step['time']}
        for component_name, states in step.items():
            if component_name == 'time' or not isinstance(states, dict): continue
            for state_key, value in states.items():
                row[f"{component_name}.{state_key}"] = value
        flat_history.append(row)
    results_df = pd.DataFrame(flat_history).set_index('time')
    
    plt.figure(figsize=(15, 7))
    plt.plot(results_df.index, results_df['data_driven_reservoir.water_level'], marker='o', linestyle='-')
    plt.title('数据驱动下水库水位的变化')
    plt.xlabel('时间 (s)')
    plt.ylabel('水位 (m)')
    plt.grid(True)
    plt.show()

plot_history(history)